In [1]:
import numpy as np
import pandas as pd

import gensim
import scipy
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [2]:
class BertToW2v(torch.nn.Module):
    def __init__(self, bert_model_name, lin_shape_in, lin_shape_out, emb_layer): # -, 768, 100, 6
        super(BertToW2v, self).__init__()
        self.emb_layer = emb_layer
        self.bert_model = BertModel.from_pretrained(bert_model_name)
        #self.bert_model.eval()
        self.linear_model = torch.nn.Linear(lin_shape_in, lin_shape_out, bias=True) # bias?
        torch.nn.init.uniform_(self.linear_model.weight, -0.1, 0.1)
        
    def forward(self, input_sentence): # ожидаем уже токенизированное предложение
        encoded_layers, _ = self.bert_model(input_sentence)
        bert_output = encoded_layers[self.emb_layer][0][1]
        linear_output = self.linear_model(bert_output).unsqueeze(0)
        return linear_output

In [3]:
ozhegov_lfreq = pd.read_csv('rus/ozhegov/ozhegov_no_emb.csv')
defs = set(ozhegov_lfreq['word'])
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
w2v = gensim.models.KeyedVectors.load_word2vec_format('w2v_models/all_norm-sz500-w10-cb0-it3-min5.w2v', binary=True, unicode_errors='ignore')

test = pd.read_csv('russe-evaluation/russe/evaluation/test.csv')

In [4]:
bw2v = BertToW2v('bert-base-multilingual-cased', lin_shape_in=768, lin_shape_out=500, emb_layer=6)
bw2v.load_state_dict(torch.load('models/SUM_5epochs_hyphen.mdl'))
bw2v.to('cuda');

In [5]:
def find_embedding(word, model, w2v, ozhegov_lfreq, defs, tokenizer):
    if word in defs:
        print(word)
        print('IN DEFS')
        defin = ozhegov_lfreq[ozhegov_lfreq['word'] == word].reset_index()['definition'][0]
        defin = '[CLS] [MASK] - ' + defin + ' [SEP]'
        tokens = tokenizer.tokenize(defin)
        tok_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokens)]).to('cuda')
        with torch.no_grad():
            embedding = model(tok_ids)
        return embedding.to('cpu').numpy()
    else:
        try:
            return w2v.get_vector(word)
        except KeyError:
            return np.nan
        
def get_cosine_distance(word1, word2, model, w2v, ozhegov_lfreq, defs, tokenizer):
    emb1 = find_embedding(word1, model, w2v, ozhegov_lfreq, defs, tokenizer)
    emb2 = find_embedding(word2, model, w2v, ozhegov_lfreq, defs, tokenizer)
    if (np.isnan(np.sum(emb1)) or np.isnan(np.sum(emb2))):
        return np.nan
    return 1 - scipy.spatial.distance.cosine(emb1, emb2)

assert abs(get_cosine_distance('абрикос', 'год', bw2v, w2v, ozhegov_lfreq, defs, tokenizer) - (1 - w2v.distance('абрикос', 'год'))) < 0.000001

In [6]:
get_cosine_distance('диск-жокей', 'зачем-то', bw2v, w2v, ozhegov_lfreq, defs, tokenizer)

диск-жокей
IN DEFS
зачем-то
IN DEFS


0.7680607438087463

In [7]:
defin = ozhegov_lfreq[ozhegov_lfreq['word'] == 'зачем-то'].reset_index()['definition'][0]
defin = '[CLS] [MASK] - ' + defin + ' [SEP]'
tokens = tokenizer.tokenize(defin)
tok_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokens)]).to('cuda')

In [8]:
with torch.no_grad():
    tmp = bw2v.bert_model(tok_ids)

In [9]:
test['sim'] = test.apply(lambda row: get_cosine_distance(row['word1'], row['word2'], bw2v, w2v, ozhegov_lfreq, defs, tokenizer), axis=1)

юго-восток
IN DEFS
северо-запад
IN DEFS
премьер-министр
IN DEFS
гамма-излучение
IN DEFS
северо-запад
IN DEFS
гран-при
IN DEFS
генерал-губернатор
IN DEFS
иван-чай
IN DEFS
иван-да-марья
IN DEFS
иван-чай
IN DEFS
иван-чай
IN DEFS
иван-чай
IN DEFS
юго-восток
IN DEFS
юго-запад
IN DEFS
кока-кола
IN DEFS
кока-кола
IN DEFS
кока-кола
IN DEFS
кока-кола
IN DEFS
премьер-министр
IN DEFS
пресс-конференция
IN DEFS
рок-н-ролл
IN DEFS
социал-демократия
IN DEFS
статс-секретарь
IN DEFS
гран-при
IN DEFS


In [10]:
test['sim'] = test['sim'].fillna(test['sim'].mean())

In [11]:
test.to_csv('russe-evaluation/russe/evaluation/SUM_model.csv', index=None)

In [12]:
ozhegov_lfreq.shape

(1209, 2)

In [13]:
ozhegov_lfreq

,word,definition
0,образцово-показательный,"образцовый и предназначенный для показа, для п..."
1,гено...,относящийся к генам
2,покапризичать,"капризничать некоторое время, слегка"
3,издохнуть.,<= д¦хнуть
4,диск-жокей,ведущий программу в дискотеке
5,куда-нибудь,безразлично куда; в неопределенное место
6,иван-да-марья,травянистое растение с желтыми цветками и фиол...
7,испакостить.,<= пакостить
8,медно-красный,"имеющий цвет меди, желто-красный"
9,"вспучить,-ся",<= пучить
